# Contents: Data Preparation

1. Environment Setup
2. Data Wrangling: Orders
3. Data Wrangling: Departments
4. Data Consistency: Products
5. Data Consistency: Orders
6. Data Merge: Orders, Orders_Prior into Orders_Products_Combined
7. Data Merge: Order_Products_Combined, Products into Orders_Products_Merged
8. Data Derivations
9. Data Aggregations
10. Data Wrangling: Customers
11. Data Merge: Orders_Products_Merged, Customers into Orders_Products_All
12. Orders_Products_All Basic Statistics

In this script we are using all of the data sets that Instacart has provided for us and cleaning, merging and preparing them for our analysis.

### 1. Environment Setup

This is where we set up our Python environment, importing the libraries that we used as well as the data sets that we are cleaning and drawing our analyses from.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
# Import data sets
path = r'C:\Users\felix\Documents\PSU\class\Instacart Basket Analysis'

df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'orders.csv'), index_col = False)
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'departments.csv'), index_col = False)
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'products.csv'), index_col = False)
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'order_products__prior.csv'), index_col = False)
customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

Below here is where we cleaned our data sets in preparation for our analysis and the creation of all the visualizations that we made, as well as where we created any additional variables that we needed for our work.

### 2. Data Wrangling: Orders

In [3]:
# Dropping irrelevant column for our project

df_ords = df_ords.drop(columns = ['eval_set'])

In [4]:
# Renaming column for clarity

df_ords.rename(columns = {'order_dow' : 'order_day_of_week'}, inplace = True)

In [5]:
df_ords.rename(columns = {'order_number' : 'user_order_number'}, inplace = True)

In [6]:
# Reformatting column type

df_ords['order_id'] = df_ords['order_id'].astype('str')

In [7]:
df_ords['user_id'] = df_ords['user_id'].astype('str')

In [8]:
# Saving our wrangled orders data frame 

#df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_wrangled.csv'))

### 3. Data Wrangling: Departments

In [9]:
# Transposing departments from horizontal to vertical

df_dep_t = df_dep.T

In [10]:
# Resetting table index

df_dep_t.reset_index()

,index,0
0,department_id,department
1,1,frozen
2,2,other
3,3,bakery
4,4,produce
5,5,alcohol
6,6,international
7,7,beverages
8,8,pets
9,9,dry goods pasta


In [11]:
# Locating the first row of df_dep_t to use as the header

new_header = df_dep_t.iloc[0]

In [12]:
# creates new data frame that includes everything from df_dep_t except the first row

df_dep_t_new = df_dep_t[1:]

In [13]:
# sets the header row as the df header

df_dep_t_new.columns = new_header

In [14]:
# Saving our wrangled departments data frame

#df_dep_t_new.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'departments_wrangled.csv'))

### 4. Data Consistency: Products

In [15]:
# checking for missing values

df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [16]:
# removes rows with missing values

df_prods_clean = df_prods[df_prods['product_name'].isnull()==False]

In [17]:
# checking for duplicate rows in df_prods_clean

df_dups = df_prods_clean[df_prods_clean.duplicated()]

In [18]:
# removes rows with duplicate values

df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [19]:
# Saving our checked products data frame

#df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'))

### 5. Data Consistency: Orders

In [20]:
# checking for mixed data types in every column

for col in df_ords.columns.tolist():
    weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords[weird]) > 0:
        print(col)

In [21]:
# checking for missing values

df_ords.isnull().sum()

order_id                       0
user_id                        0
user_order_number              0
order_day_of_week              0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [22]:
# imputing the missing values

df_ords['days_since_prior_order'].fillna(1.111484e+01, inplace=True)

In this step we imputed the calculated mean value of days_since_prior_order into all the missing values. We did this because there were too many rows with missing values that I felt if we removed them it might have an impact on our final results. Since there weren't many meaningful outliers in this column, I thought that imputing the missing values would be acceptable in this case.

In [23]:
# checking for rows with duplicate values

df_ords[df_ords.duplicated()]

,order_id,user_id,user_order_number,order_day_of_week,order_hour_of_day,days_since_prior_order


In [24]:
# Saving our checked orders data frame

#df_ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'))

### 6. Data Merge: Orders, Orders_Prior into Orders_Products_Combined

In [25]:
# changing column type for merge

df_ords_prior['order_id'] = df_ords_prior['order_id'].astype('str')

In [26]:
# default inner join

df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id')

In [27]:
# Saving our merged orders data frame

#df_merged_large.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

### 7. Data Merge: Order_Products_Combined, Products into Orders_Products_Merged

In [28]:
# merging with products

df_merge = df_merged_large.merge(df_prods_clean_no_dups, on = 'product_id')

In [29]:
# Saving our merged orders and products data frame

#df_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged.pkl'))

### 8. Data Derivations

In [30]:
# creating new variable price_range_loc based on the range of prices

df_merge.loc[df_merge['prices'] > 15, 'price_range_loc'] = 'High-range product'

In [31]:
df_merge.loc[(df_merge['prices'] <= 15) & (df_merge['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [32]:
df_merge.loc[df_merge['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [33]:
# creating new variable busiest_day using for loop of order_day_of_week

result = []

for value in df_merge['order_day_of_week']:
    if value == 0:
        result.append('Busiest day')
    elif value == 4:
        result.append('Least busy')
    else:
        result.append('Regularly busy')

In [34]:
df_merge['busiest_day'] = result

In [35]:
# creating new variable busiest_days using for loop of order_day_of week
# instead of a singular busiest day, this uses the 2 busiest and 2 least busy days

results2 = []

for value in df_merge['order_day_of_week']:
    if (value == 0) or (value == 1):
        results2.append('Busier days')
    elif (value == 4) or (value == 3):
        results2.append('Less busy days')
    else:
        results2.append('Regularly busy')

In [36]:
df_merge['busiest_days'] = results2

In [37]:
# creating new variable busiest_period_of_day using for loop of order_hour_of_day

results3 = []

for value in df_merge['order_hour_of_day']:
    if value in [10, 11, 14, 15, 13, 12, 16, 9]:
        results3.append('Most orders')
    elif value in [3, 4, 2, 5, 1, 0, 6, 23]:
        results3.append('Fewest orders')
    else:
        results3.append('Average orders')

In [38]:
df_merge['busiest_period_of_day'] = results3

In [39]:
# Saving our data frame with new variables

#df_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged2.pkl'))

### 9. Data Aggregations

In [40]:
# creating column max_order by an aggregation using transform()

df_merge['max_order'] = df_merge.groupby(['user_id'])['user_order_number'].transform(np.max)

In [41]:
# creating new variable loyalty_flag using loc() with max_order

df_merge.loc[df_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [42]:
df_merge.loc[(df_merge['max_order'] <= 40) & (df_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [43]:
df_merge.loc[df_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [44]:
# creating column mean_prices_products by an aggregation using transform()

df_merge['mean_prices_products'] = df_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [45]:
# creating new variable spender using loc() wtih mean_prices_products

df_merge.loc[df_merge['mean_prices_products'] < 10, 'spender'] = 'Low spender'

In [46]:
df_merge.loc[df_merge['mean_prices_products'] >= 10, 'spender'] = 'High spender'

In [47]:
# creating coluimn median_days_last_order by an aggregation using transform()

df_merge['median_days_last_order'] = df_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [48]:
# creating new variable freq_customer using loc() with median_days_last_order

df_merge.loc[df_merge['median_days_last_order'] > 20, 'freq_customer'] = 'Non-frequent customer'

In [49]:
df_merge.loc[(df_merge['median_days_last_order'] <= 20) & (df_merge['median_days_last_order'] > 10), 'freq_customer'] = 'Regular customer'

In [50]:
df_merge.loc[df_merge['median_days_last_order'] <= 10, 'freq_customer'] = 'Frequent customer'

In [51]:
# Saving our data frame with new variables

#df_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged3.pkl'))

### 10. Data Wrangling: Customers

In [52]:
# renaming columns for clarity

customer.rename(columns = {'Surnam': 'Last Name', 'STATE': 'State'}, inplace = True)

In [53]:
# reformatting column type 

customer['user_id'] = customer['user_id'].astype('str')

In [54]:
# checking for mixed data types

for col in customer.columns.tolist():
    weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
    if len(customer[weird]) > 0:
        print(col)

First Name


In [55]:
# reformatting column type

customer['First Name'] = customer['First Name'].astype('str')

In [56]:
# checking for missing values

customer.isnull().sum()

user_id         0
First Name      0
Last Name       0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [57]:
# checking for duplicate rows

customer[customer.duplicated()]

,user_id,First Name,Last Name,Gender,State,Age,date_joined,n_dependants,fam_status,income


### 11. Data Merge: Orders_Products_Merged, Customers into Orders_Products_All

In [58]:
# converting user_id in ords_prods_merge to str type before combining data frames

df_merge['user_id'] = df_merge['user_id'].astype('str')

In [59]:
# combining the data frames

ords_prods_all = df_merge.merge(customer, on = 'user_id')

In [60]:
# Removing First Name and Last Name from file due to PII

ords_prods_all_2 = ords_prods_all.drop(columns = ['First Name', 'Last Name'])

In [61]:
# Saving our merged data frame
 
#ords_prods_all_2.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))

### 12. Orders_Products_All Basic Statistics

In [62]:
ords_prods_all_2.head()

,order_id,user_id,user_order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spender,median_days_last_order,freq_customer,Gender,State,Age,date_joined,n_dependants,fam_status,income
0,2539329,1,1,2,8,11.11484,196,1,0,Soda,...,Low spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.00000,196,1,1,Soda,...,Low spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.00000,196,1,1,Soda,...,Low spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.00000,196,1,1,Soda,...,Low spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.00000,196,1,1,Soda,...,Low spender,20.0,Regular customer,Female,Alabama,31,2/17/2019,3,married,40423


In [63]:
ords_prods_all_2.shape

(32404859, 30)

In [64]:
ords_prods_all_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 30 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                object 
 1   user_id                 object 
 2   user_order_number       int64  
 3   order_day_of_week       int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
 6   product_id              int64  
 7   add_to_cart_order       int64  
 8   reordered               int64  
 9   product_name            object 
 10  aisle_id                int64  
 11  department_id           int64  
 12  prices                  float64
 13  price_range_loc         object 
 14  busiest_day             object 
 15  busiest_days            object 
 16  busiest_period_of_day   object 
 17  max_order               int64  
 18  loyalty_flag            object 
 19  mean_prices_products    float64
 20  spender                 object 
 21  median_days_last_order  float

In [65]:
ords_prods_all_2.describe()

,user_order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices,max_order,mean_prices_products,median_days_last_order,Age,n_dependants,income
count,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07,3.240486e+07
mean,1.714230e+01,2.738867e+00,1.342515e+01,1.110477e+01,2.559866e+04,8.352547e+00,5.895873e-01,7.119612e+01,9.919792e+00,1.198023e+01,3.305217e+01,1.198023e+01,1.002895e+01,4.946527e+01,1.501896e+00,9.943773e+04
std,1.753532e+01,2.090077e+00,4.246380e+00,8.493183e+00,1.408400e+04,7.127071e+00,4.919087e-01,3.821139e+01,6.281485e+00,4.956554e+02,2.515525e+01,8.324227e+01,6.324929e+00,1.848558e+01,1.118865e+00,4.305727e+04
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.800000e+01,0.000000e+00,2.590300e+04
25%,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.354400e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.200000e+00,1.300000e+01,7.387298e+00,6.000000e+00,3.300000e+01,1.000000e+00,6.700400e+04
50%,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.530200e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,7.400000e+00,2.600000e+01,7.824786e+00,8.000000e+00,4.900000e+01,2.000000e+00,9.661800e+04
75%,2.400000e+01,5.000000e+00,1.600000e+01,1.400000e+01,3.794700e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.130000e+01,4.700000e+01,8.254023e+00,1.200000e+01,6.500000e+01,3.000000e+00,1.279120e+05
max,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,9.999900e+04,9.900000e+01,2.500542e+04,3.000000e+01,8.100000e+01,3.000000e+00,5.939010e+05
